In [1]:
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from imutils import paths
import random
import cv2
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import flwr as fl
# Load and compile Keras model


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
BATCH_SIZE = 8
train_data_dir = 'D:/TQ/Federated/5/train/'

test_data_dir = 'D:/TQ/Federated/5/val/'


In [4]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split percentage
)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)




In [5]:
print("Class Counts in Training Samples:")
total_classes = sorted(os.listdir(train_data_dir))
print(total_classes)
for category in total_classes:
    category_path = os.path.join(train_data_dir, category)
    sample_count = len(os.listdir(category_path))
    print(f"{category}: {sample_count} samples")


Class Counts in Training Samples:
['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
Animal_Abuse: 168 samples
Arson: 101 samples
Fight: 88 samples
Normal: 199 samples
Riot: 418 samples
Traffic_Accident: 138 samples
Train_Accident: 0 samples


In [6]:
train_classes = sorted(os.listdir(train_data_dir))
test_classes = sorted(os.listdir(test_data_dir))


In [7]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the base model layers
base_model.trainable = False

# Add custom classification layers on top of the base model
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Add dropout layer with a dropout rate of 0.5
outputs = Dense(7, activation='softmax')(x)  # Assuming binary classification

# Create the full model
model5 = Model(inputs, outputs)
optimizer = Adam(learning_rate=0.001)
model5.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [8]:
traingen = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(225, 225),
                                                   class_mode='categorical',
                                                   classes=total_classes,
                                                   subset='training',
                                                   batch_size=BATCH_SIZE, 
                                                   shuffle=True,
                                                   seed=42)

validgen = train_datagen.flow_from_directory(train_data_dir,
                                               target_size=(225, 225),
                                               class_mode='categorical',
                                               classes=total_classes,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_datagen.flow_from_directory(test_data_dir,
                                             target_size=(225, 225),
                                             class_mode='categorical',
                                             classes=total_classes,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)


Found 893 images belonging to 7 classes.
Found 219 images belonging to 7 classes.
Found 292 images belonging to 7 classes.


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
callbacks = [early_stopping]


In [12]:
history = model5.fit(
            traingen,
            epochs=5,
            steps_per_epoch=len(traingen),
            validation_data=validgen,
            validation_steps=len(validgen),
            callbacks=callbacks,
            verbose = 1
)

Epoch 1/5


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


112/112 [==============================] - 18s 137ms/step - loss: 0.8511 - accuracy: 0.7044 - val_loss: 1.2548 - val_accuracy: 0.5479
Epoch 2/5
112/112 [==============================] - 14s 123ms/step - loss: 0.2333 - accuracy: 0.9239 - val_loss: 1.7190 - val_accuracy: 0.4338
Epoch 3/5
112/112 [==============================] - 13s 120ms/step - loss: 0.1592 - accuracy: 0.9462 - val_loss: 1.5053 - val_accuracy: 0.5525
Epoch 4/5
112/112 [==============================] - 14s 123ms/step - loss: 0.1067 - accuracy: 0.9642 - val_loss: 1.5390 - val_accuracy: 0.5890
Epoch 5/5
112/112 [==============================] - 16s 142ms/step - loss: 0.0944 - accuracy: 0.9698 - val_loss: 1.7063 - val_accuracy: 0.5662


In [13]:
#model = tf.keras.models.load_model("D:/TQ/Federated/Client5_Centralized.h5")

In [14]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = model5.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

292/292 [==============================] - 6s 18ms/step


In [15]:
print("Accuracy:")
test_loss, test_accuracy = model5.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Accuracy:
Test Loss: 0.6739577054977417
Test Accuracy: 0.9041095972061157


In [16]:
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Confusion Matrix Before Aggregation:
[[40  0  0  2  0  0  0]
 [ 0 25  0  1  0  0  0]
 [ 0  0 22  0  0  0  0]
 [ 0  2  0 44  0  4  0]
 [ 0  1  0  6 98  0  0]
 [ 0  0  0  0  0 35  0]
 [ 0  0  0 12  0  0  0]]


In [17]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm2, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix5.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [24]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm2[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm2[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 40, Total Instances: 42
Arson - True Positives: 26, Total Instances: 26
Fight - True Positives: 22, Total Instances: 22
Normal - True Positives: 44, Total Instances: 50
Riot - True Positives: 99, Total Instances: 105
Traffic_Accident - True Positives: 35, Total Instances: 35
Train_Accident - True Positives: 0, Total Instances: 12
{'Animal_Abuse': 0.9523809523809523, 'Arson': 1.0, 'Fight': 1.0, 'Normal': 0.88, 'Riot': 0.9428571428571428, 'Traffic_Accident': 1.0, 'Train_Accident': 0.0}


In [18]:
model5.save("Client5_Centralized.h5")

In [19]:
client5_center = tf.keras.models.load_model('Client5_Centralized.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [20]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client5_center.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


292/292 [==============================] - 5s 14ms/step


In [21]:
print("Accuracy:")
test_loss, test_accuracy = client5_center.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
# Calculate the confusion matrix
cm2 = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm2)

Accuracy:
Test Loss: 0.6849393248558044
Test Accuracy: 0.9075342416763306
Confusion Matrix Before Aggregation:
[[40  0  0  2  0  0  0]
 [ 0 26  0  0  0  0  0]
 [ 0  0 22  0  0  0  0]
 [ 0  2  0 44  0  4  0]
 [ 0  3  0  3 99  0  0]
 [ 0  0  0  0  0 35  0]
 [ 0  0  0 11  1  0  0]]


In [22]:
from sklearn.metrics import classification_report 
print(classification_report(y_true, y_pred_labels, target_names=total_classes))


                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.95      0.98        42
           Arson       0.84      1.00      0.91        26
           Fight       1.00      1.00      1.00        22
          Normal       0.73      0.88      0.80        50
            Riot       0.99      0.94      0.97       105
Traffic_Accident       0.90      1.00      0.95        35
  Train_Accident       0.00      0.00      0.00        12

        accuracy                           0.91       292
       macro avg       0.78      0.83      0.80       292
    weighted avg       0.88      0.91      0.89       292



C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [26]:
client5 = tf.keras.models.load_model('client5_30_fedavg.h5')

C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\random.py:370: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return self.randrange(a, b+1)


In [27]:
y_true = testgen.classes

# Calculate predictions on the test data
y_pred = client5.predict(testgen)
y_pred_labels = np.argmax(y_pred, axis=1)  # Get the predicted class labels
test_loss, test_accuracy = client5.evaluate(testgen, steps=len(testgen), verbose=0)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print(classification_report(y_true, y_pred_labels, target_names=total_classes))


C:\Users\cslen\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\preprocessing\image.py:2616: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


292/292 [==============================] - 5s 15ms/step
Test Loss: 0.44485604763031006
Test Accuracy: 0.9109588861465454
                  precision    recall  f1-score   support

    Animal_Abuse       1.00      0.98      0.99        42
           Arson       0.95      0.81      0.88        26
           Fight       1.00      1.00      1.00        22
          Normal       0.72      0.92      0.81        50
            Riot       1.00      0.96      0.98       105
Traffic_Accident       0.88      1.00      0.93        35
  Train_Accident       1.00      0.17      0.29        12

        accuracy                           0.92       292
       macro avg       0.94      0.83      0.84       292
    weighted avg       0.93      0.92      0.91       292



In [28]:
cm = confusion_matrix(y_true, y_pred_labels)
print("Confusion Matrix Before Aggregation:")
print(cm)

Confusion Matrix Before Aggregation:
[[ 41   0   0   1   0   0   0]
 [  0  21   0   4   0   1   0]
 [  0   0  22   0   0   0   0]
 [  0   0   0  46   0   4   0]
 [  0   1   0   3 101   0   0]
 [  0   0   0   0   0  35   0]
 [  0   0   0  10   0   0   2]]


In [29]:
import pandas as pd
from openpyxl import Workbook

categories = ['Animal_Abuse', 'Arson', 'Fight', 'Normal', 'Riot', 'Traffic_Accident', 'Train_Accident']
df_cm = pd.DataFrame(cm, index=categories, columns=categories)

# Write the confusion matrix DataFrame to an Excel file
excel_writer = pd.ExcelWriter('D:/TQ/Federated/confusion_matrix.xlsx')
# Create an Excel workbook and write the confusion matrix DataFrame to a sheet
wb = Workbook()
ws = wb.active
ws.title = 'Confusion Matrix'

# Write the column names
ws.append([''] + df_cm.columns.tolist())

# Write the confusion matrix data
for index, row in df_cm.iterrows():
    ws.append([index] + row.tolist())

# Save the workbook to an Excel file
wb.save('D:/TQ/Federated/confusion_matrix5f.xlsx')

print("Confusion matrix written to confusion_matrix.xlsx")

Confusion matrix written to confusion_matrix.xlsx


In [30]:
from sklearn.metrics import confusion_matrix
import numpy as np


# We will store the results in a dictionary for easy access later
per_class_accuracies = {}

# Calculate the accuracy for each one of our classes
for idx, cls in enumerate(total_classes):
    true_positives = cm[idx, idx]
    
    # Total instances of the current class
    total_instances = np.sum(cm[idx, :])
    
    # The accuracy for the current class is the ratio between correct predictions to all predictions
    per_class_accuracies[cls] = true_positives / total_instances if total_instances > 0 else 0
    print(f"{cls} - True Positives: {true_positives}, Total Instances: {total_instances}")

print(per_class_accuracies)

Animal_Abuse - True Positives: 41, Total Instances: 42
Arson - True Positives: 21, Total Instances: 26
Fight - True Positives: 22, Total Instances: 22
Normal - True Positives: 46, Total Instances: 50
Riot - True Positives: 101, Total Instances: 105
Traffic_Accident - True Positives: 35, Total Instances: 35
Train_Accident - True Positives: 2, Total Instances: 12
{'Animal_Abuse': 0.9761904761904762, 'Arson': 0.8076923076923077, 'Fight': 1.0, 'Normal': 0.92, 'Riot': 0.9619047619047619, 'Traffic_Accident': 1.0, 'Train_Accident': 0.16666666666666666}
